In [1]:
import bindome as bd
import anndata as ad
import scanpy as sc
import scglue
import pandas as pd
keys=['rna', 'atac']
import networkx as nx
import scglue
#define adatas
rna1 = ad.read_h5ad("rna-pp2.h5ad")

atac1 = ad.read_h5ad("atac-pp2.h5ad")
atac1=atac1[pd.Series(atac1.obs_names).sample(1000),pd.Series(atac1.var_names).sample(1000)].copy()
#
scglue.models.configure_dataset(
    rna1, "NB", use_highly_variable=True,
    use_layer="counts", use_rep="X_pca"
)

scglue.models.configure_dataset(
    atac1, "NB", use_highly_variable=True,
    use_rep="X_lsi"
)
guidance = nx.read_graphml("guidance2.graphml.gz")

adatas={"rna": rna1, "atac": atac1}
pretrain_init_kws={}
pretrain_init_kws.update({"shared_batches": False})


pretrain = scglue.models.scglue.SCGLUEModel(adatas,sorted(guidance.nodes),**pretrain_init_kws)
modalities=pretrain.modalities
from scglue.models.data import AnnDataset, ArrayDataset, DataLoader, GraphDataset


anndata = AnnDataset(
            [adatas[key] for key in keys],
            [modalities[key] for key in keys],
            mode="train"
        )

data_loader = DataLoader( #three elements: x, xrep, _
            anndata, batch_size=128,
            shuffle=False, drop_last=False
        )

[INFO] autodevice: Using CPU as computation device.


In [285]:
modalities.keys(),modalities['atac']['use_rep'],atac1.obsm['X_lsi']

(dict_keys(['rna', 'atac']),
 'X_lsi',
 array([[ 1.092738  , -0.55591387, -0.18498696, ..., -0.04381248,
         -1.3174046 , -0.7454044 ],
        [ 0.82954   , -1.5036018 ,  0.36917016, ..., -1.2883828 ,
         -1.2940404 ,  1.3559904 ],
        [ 0.8692328 ,  0.4350031 , -0.03554815, ..., -1.7968197 ,
          0.77023846, -0.7731513 ],
        ...,
        [ 1.2505072 ,  0.18474104, -0.78234136, ..., -1.6482024 ,
         -0.04877033, -0.47360432],
        [ 1.4429551 , -0.39928216,  1.0854658 , ..., -1.1394583 ,
          0.6945665 , -0.02570789],
        [ 0.9694253 ,  1.4038382 , -1.5304313 , ...,  1.0953511 ,
          0.2768458 , -0.72715276]], dtype=float32))

In [288]:
xrep=atac1.obsm['X_lsi']
xrep.shape,modalities['atac']['rep_dim']

((1000, 100), 100)

In [290]:
np.empty((100, 0), dtype=np.float)

/var/folders/12/d54y_pk95zdbftjg_qkcj_c80000gq/T/ipykernel_54883/3363109874.py:1: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np.empty((100, 0), dtype=np.float)


array([], shape=(100, 0), dtype=float64)

In [3]:
modalities['rna'].keys()

dict_keys(['prob_model', 'use_highly_variable', 'features', 'use_layer', 'use_rep', 'rep_dim', 'use_batch', 'batches', 'use_cell_type', 'cell_types', 'use_dsc_weight', 'use_obs_names'])

In [4]:
import torch
for data in anndata:
#     print(data[0:2])
    out=tuple(map(lambda x: torch.cat(x, dim=0), zip(data)))
    break
out

(tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0.

In [5]:
iter1 = torch.tensor([1, 2, 3])
iter2 = torch.tensor([4, 5, 6])
iter3 = torch.tensor([7, 8, 9])

data = [iter1, iter2, iter3]

transposed = zip(data)
for item in transposed:
    print(item)
    
transposed = zip(*data)
out1=tuple(map(lambda x: torch.cat(x, dim=0), zip(data)))
print(out1)
data=[[iter1],[iter2],[iter3]]
print(tuple(map(lambda x: torch.cat(x, dim=0), zip(*data))))
for item in transposed:
    print(item)

(tensor([1, 2, 3]),)
(tensor([4, 5, 6]),)
(tensor([7, 8, 9]),)
(tensor([1, 2, 3]), tensor([4, 5, 6]), tensor([7, 8, 9]))
(tensor([1, 2, 3, 4, 5, 6, 7, 8, 9]),)
(tensor(1), tensor(4), tensor(7))
(tensor(2), tensor(5), tensor(8))
(tensor(3), tensor(6), tensor(9))


In [6]:
for data in data_loader:
#     print(data[0:2])
    out=tuple(map(lambda x: torch.cat(x, dim=0), zip(data)))
    break
out

(tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 tensor([[ 3.4848e+00,  1.1419e+01,  9.0474e+00,  ..., -5.3635e+00,
           1.6005e-01, -1.2033e+00],
         [ 1.9448e-01, -4.5760e-01,  1.2121e+00,  ...,  7.5727e-02,
          -1.0081e-02, -4.6408e-02],
         [-1.4876e+00, -2.0016e+00,  1.0567e+00,  ..., -2.0454e-01,
           2.3666e-01, -3.0249e-01],
         ...,
         [-1.8596e+00, -1.6021e+00,  7.4491e-02,  ...,  1.5663e-01,
           7.0951e-01,  2.2479e-01],
         [ 4.1313e-02,  6.0275e+00, -8.897

In [7]:
out[0:2]# count data for rna and atac

(tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]))

In [8]:
for data in data_loader:
    out=data_loader._collate([data])
    break
print(out[0:2])
x_rna=out[0:2][0]
x_atac=out[0:2][1]
x_rna.shape,x_atac.shape

(tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]))


(torch.Size([128, 800]), torch.Size([128, 57]))

In [9]:
#batch 128 check if rna and atac count data always have same length in each batch
#if convert atac into seq there wull be different length in one batch 
for data in data_loader:
    out=data_loader._collate([data])
    x_rna=out[0:2][0]
    x_atac=out[0:2][1]
    print(x_rna.shape,x_atac.shape)
    

torch.Size([128, 800]) torch.Size([128, 57])
torch.Size([128, 800]) torch.Size([128, 57])
torch.Size([128, 800]) torch.Size([128, 57])
torch.Size([128, 800]) torch.Size([128, 57])
torch.Size([128, 800]) torch.Size([128, 57])
torch.Size([128, 800]) torch.Size([128, 57])
torch.Size([128, 800]) torch.Size([128, 57])
torch.Size([128, 800]) torch.Size([128, 57])
torch.Size([128, 800]) torch.Size([128, 57])
torch.Size([128, 800]) torch.Size([128, 57])
torch.Size([128, 800]) torch.Size([128, 57])
torch.Size([128, 800]) torch.Size([128, 57])
torch.Size([128, 800]) torch.Size([128, 57])
torch.Size([128, 800]) torch.Size([128, 57])
torch.Size([128, 800]) torch.Size([128, 57])
torch.Size([80, 800]) torch.Size([80, 57])


In [ ]:
#use bindome to get atac seq 
import bindome
mypath='mm10.fa'
seq2=bindome.tl.get_sequences_from_bed(atac1.var[['chrom', 'chromStart', 'chromEnd']].head(1000), genome='mm10', uppercase=True,gen_path=mypath)
seq2

/var/folders/12/d54y_pk95zdbftjg_qkcj_c80000gq/T/tmpzmpkbmaw
genome mm10 True
mm10.fa
True mm10.fa
running bedtools...
bedtools getfasta -fi mm10.fa -bed /var/folders/12/d54y_pk95zdbftjg_qkcj_c80000gq/T/tmpzmpkbmaw -fo /var/folders/12/d54y_pk95zdbftjg_qkcj_c80000gq/T/tmpvbbsmn8p


In [ ]:
len(seq2)

In [ ]:
# atac1.shape

In [13]:
atac1.shape

(1000, 1000)

In [14]:
atac1.X.toarray().shape

(1000, 1000)

In [15]:
import numpy as np
chrom_num=len(seq2)

def dna_1hot(seq, seq_len=None, n_uniform=False):
    """dna_1hot
    Args:
      seq:       nucleotide sequence.
      seq_len:   length to extend/trim sequences to.
      n_uniform: represent N's as 0.25, forcing float16,
                 rather than sampling.
    Returns:
      seq_code: length by nucleotides array representation.
    """
    if seq_len is None:
        seq_len = len(seq)
        seq_start = 0
    else:
        if seq_len <= len(seq):
            # trim the sequence
            seq_trim = (len(seq) - seq_len) // 2
            seq = seq[seq_trim : seq_trim + seq_len]
            seq_start = 0
        else:
            seq_start = (seq_len - len(seq)) // 2
    seq = seq.upper()

    # map nt's to a matrix len(seq)x4 of 0's and 1's.
    if n_uniform:
        seq_code = np.zeros((seq_len, 4), dtype="float16")
    else:
        seq_code = np.zeros((seq_len, 4), dtype="int")

    for i in range(seq_len):
        if i >= seq_start and i - seq_start < len(seq):
            nt = seq[i - seq_start]
            if nt == "A":
                seq_code[i, 0] = 1
            elif nt == "C":
                seq_code[i, 1] = 1
            elif nt == "G":
                seq_code[i, 2] = 1
            elif nt == "T":
                seq_code[i, 3] = 1
            else:
                if n_uniform:
                    seq_code[i, :] = 0.25
                else:
                    ni = random.randint(0, 3)
                    seq_code[i, ni] = 1
    return seq_code
count=0
list_seq=[]
for i in range(chrom_num):
    count+=1
    onehotseq=dna_1hot(seq2[i][1])
    print(onehotseq.shape)
    list_seq.append(seq2[i][1])
#     list_seq.append(torch.tensor(onehotseq))
print(count)
# current x shape 1000,4,???(not same size) pad zero/ truncate for certain length 
# how to replace with the part anndata.X?

(507, 4)
(542, 4)
(427, 4)
(424, 4)
(226, 4)
(281, 4)
(700, 4)
(295, 4)
(491, 4)
(336, 4)
(789, 4)
(361, 4)
(264, 4)
(149, 4)
(241, 4)
(182, 4)
(319, 4)
(172, 4)
(896, 4)
(601, 4)
(457, 4)
(292, 4)
(370, 4)
(361, 4)
(328, 4)
(531, 4)
(224, 4)
(304, 4)
(486, 4)
(292, 4)
(509, 4)
(209, 4)
(215, 4)
(234, 4)
(246, 4)
(305, 4)
(303, 4)
(557, 4)
(1167, 4)
(545, 4)
(485, 4)
(277, 4)
(335, 4)
(320, 4)
(183, 4)
(244, 4)
(561, 4)
(573, 4)
(206, 4)
(234, 4)
(219, 4)
(657, 4)
(449, 4)
(206, 4)
(396, 4)
(256, 4)
(328, 4)
(305, 4)
(274, 4)
(477, 4)
(318, 4)
(374, 4)
(153, 4)
(413, 4)
(361, 4)
(197, 4)
(238, 4)
(219, 4)
(212, 4)
(787, 4)
(308, 4)
(419, 4)
(337, 4)
(818, 4)
(234, 4)
(269, 4)
(355, 4)
(239, 4)
(190, 4)
(247, 4)
(363, 4)
(362, 4)
(189, 4)
(344, 4)
(149, 4)
(391, 4)
(302, 4)
(200, 4)
(399, 4)
(250, 4)
(477, 4)
(303, 4)
(330, 4)
(825, 4)
(393, 4)
(344, 4)
(284, 4)
(225, 4)
(265, 4)
(483, 4)
(194, 4)
(485, 4)
(375, 4)
(255, 4)
(297, 4)
(699, 4)
(519, 4)
(279, 4)
(454, 4)
(201, 4)
(217, 4)


In [16]:
list_seq

['...']

In [17]:
for data in anndata:
    print(data)
    print(len(data))
    break

[tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0.

In [18]:
anndata.data_configs[0].keys()

dict_keys(['prob_model', 'use_highly_variable', 'features', 'use_layer', 'use_rep', 'rep_dim', 'use_batch', 'batches', 'use_cell_type', 'cell_types', 'use_dsc_weight', 'use_obs_names'])

In [19]:
anndata.data_configs[1].keys()

dict_keys(['prob_model', 'use_highly_variable', 'features', 'use_layer', 'use_rep', 'rep_dim', 'use_batch', 'batches', 'use_cell_type', 'cell_types', 'use_dsc_weight', 'use_obs_names'])

In [20]:
anndata.data_configs[1]['use_obs_names']

False

In [21]:
rna1

AnnData object with n_obs × n_vars = 1000 × 28930
    obs: 'domain', 'cell_type'
    var: 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm', 'mean', 'std', 'chrom', 'chromStart', 'chromEnd', 'name', 'score', 'strand', 'thickStart', 'thickEnd', 'itemRgb', 'blockCount', 'blockSizes', 'blockStarts', 'gene_id', 'gene_type', 'mgi_id', 'havana_gene', 'tag'
    uns: 'hvg', 'log1p', 'neighbors', 'pca', 'umap', '__scglue__'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    layers: 'counts'
    obsp: 'connectivities', 'distances'

In [22]:
atac1

AnnData object with n_obs × n_vars = 1000 × 1000
    obs: 'domain', 'cell_type'
    var: 'chrom', 'chromStart', 'chromEnd', 'highly_variable'
    uns: '__scglue__'
    obsm: 'X_lsi'

In [23]:
pretrain.net

SCGLUE(
  (g2v): GraphEncoder(
    (conv): GraphConv()
    (loc): Linear(in_features=50, out_features=50, bias=True)
    (std_lin): Linear(in_features=50, out_features=50, bias=True)
  )
  (v2g): GraphDecoder()
  (x2u): ModuleDict(
    (rna): NBDataEncoder(
      (linear_0): Linear(in_features=100, out_features=256, bias=True)
      (act_0): LeakyReLU(negative_slope=0.2)
      (bn_0): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (dropout_0): Dropout(p=0.2, inplace=False)
      (linear_1): Linear(in_features=256, out_features=256, bias=True)
      (act_1): LeakyReLU(negative_slope=0.2)
      (bn_1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (dropout_1): Dropout(p=0.2, inplace=False)
      (loc): Linear(in_features=256, out_features=50, bias=True)
      (std_lin): Linear(in_features=256, out_features=50, bias=True)
    )
    (atac): NBDataEncoder(
      (linear_0): Linear(in_features=100, out_features=

In [24]:
import mubind


In [25]:
first_chrom=seq2[0][1]

In [26]:
#mubind conversion to onehot 
mb_onehot=mubind.mubind.tl.encoding.onehot_mononuc(first_chrom)


/Users/meiqiliu/miniforge3/envs/fa_base2/lib/python3.8/site-packages/mubind/tl/encoding.py:79: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "onehot_mononuc" failed type inference due to: Internal error at <numba.core.typeinfer.ArgConstraint object at 0x32cf22400>.
Cannot determine Numba type of <class 'sklearn.preprocessing._label.LabelEncoder'>
During: typing of argument at /Users/meiqiliu/miniforge3/envs/fa_base2/lib/python3.8/site-packages/mubind/tl/encoding.py (81)
Enable logging at debug level for details.

File "../../../../../miniforge3/envs/fa_base2/lib/python3.8/site-packages/mubind/tl/encoding.py", line 81:
def onehot_mononuc(seq, label_encoder=LabelEncoder(), onehot_encoder=OneHotEncoder(sparse=False)):
    seq_arr = np.array(list(seq + "ACGNT"))
    ^

  @jit
/Users/meiqiliu/miniforge3/envs/fa_base2/lib/python3.8/site-packages/numba/core/object_mode_passes.py:151: NumbaWarning: Function "onehot_mononuc" was compiled in 

In [39]:
mb_onehot.shape

(4, 507)

In [28]:
dna_1hot(first_chrom).shape

(507, 4)

In [ ]:
for i,j in seq2.items():
    print(i)

In [90]:
seq_dict={}
for i in range(chrom_num):
    chrom_name=seq2[i][0]
    seq_dict[i]=seq2[i][1]


In [91]:
seq_dict

{0: 'GAAATCCCTTTTTAGGGATTTGTTCTTCCAGCTTTCGTCTTCCTTTTCTTGTTCTCCCTATTTCTAGATTTCTGAGAACTGTACTGTTCATTGTTTCTACCCATCATTAAGAGAAAAGATATGGAAACATTCCCATGGTAACAGCTTCATCTCAAAATATCTGAATGCTCTCAGCTGCCCTCCTACTCCAGGGAAAAGATGAGCTAGCTGCTCCCTCCCCATCCTCTCAGAGTACAGCCGACTGCTCCATCCCATACTGCAGTGAGGCTGCAGACCAGGAGCAGAGAGAATCCCAGGGAGCTAGATTAGTTTCTTAGCTGGCAGAAGCAGCAAGAACATTTCAAAATTTCAGTATTGCACATCTGACACTGGGCCTGAGAACAGCTGGTGAATAGGCACAGGGAGATCAGATGTACATAATACAGGCAGCACAGACTTGCTTCAAAGACCCACAGGTGCCACACTGAGTAATGGCAGAGAAACGGGTTTGCAAAAGTTCAGAAGTGT',
 1: 'GAGGTGTATGAAAATCTTGGGCACTCTCCAGCATCCCCAGCTGATACTGAGAGAACAGTTAAAGTGCTGAGCCCTCCTCACTGATCACAAAGTAGACTTTAATTAGTAGGAGCGGCTGTGCTCACCCAGTAGGGTGCCGGGGAGTCTGCTTCAGAAATAAAACCTCCAGCACAGAAAAGGAGAAAAGGGGCAGCTGGCGCCAGATGGTGTGGAGCTTTAGGTGGTGTGCAGTGAGCCATTTCTGACTCAAGAAGGTCTCATAGGGAGCTGATCTGTGCTAGGCGCCGCTGCCCCAGGGCAGCCCCTGGCTTTCTGTTCAGGGACCTCTGTAGCAAACGTTTCTCTAGTAGTTCTCTGCCAAAGAAGAGGCTTGGACGGTTCCTCATTGGTTAGGAAGCAGAGTGGGAGATCTTTTAGATCTCCTGAAAAGACTGCAGACAGAGGAGTTCTGGAAAAACACTGAGTGTGGGCATCTCGAAG

In [97]:
pad_seq=mubind.mubind.tl.encoding.onehot_mononuc_multi(seq_dict,700) #cannot direcly apply

IndexError: index 700 is out of bounds for axis 2 with size 700

In [52]:
pad_seq.shape

(1000, 4, 1000)

In [53]:
pad_seq[0].shape

(4, 1000)

In [94]:
def onehot_mononuc_multi(seqs, max_length):
    result = np.full([len(seqs), 4, max_length], 0.25, dtype=np.float32)
    for i, seq in seqs.items():
        print(i)
        shift = (max_length - len(seq))
        print(shift)
        print(len(seq))
        for j in range(len(seq)):
            base = seq[j]
            print(base,j)
            
            if base == "A":
                result[i, :, j + shift] = [1, 0, 0, 0]
            elif base == "C":
                result[i, :,j + shift] = [0, 1, 0, 0]
            elif base == "G":
                print(result[i, :, :])
                result[i, :, j + shift] = [0, 0, 1, 0]
            elif base == "T":
                result[i, :, j + shift] = [0, 0, 0, 1]
    return result
res=onehot_mononuc_multi(seq_dict,1000)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [82]:
a=np.full([1000, 4, 1000], 0.25, dtype=np.float32)


In [84]:
a[0, :, 3]= [1, 0, 0, 0]

In [86]:
a[0, :, 3]

array([1., 0., 0., 0.], dtype=float32)

In [98]:
res.shape

(1000, 4, 1000)

In [100]:
torch.as_tensor(res)

tensor([[[0.2500, 0.2500, 0.2500,  ..., 0.0000, 0.0000, 0.0000],
         [0.2500, 0.2500, 0.2500,  ..., 0.0000, 0.0000, 0.0000],
         [0.2500, 0.2500, 0.2500,  ..., 0.0000, 1.0000, 0.0000],
         [0.2500, 0.2500, 0.2500,  ..., 1.0000, 0.0000, 1.0000]],

        [[0.2500, 0.2500, 0.2500,  ..., 0.0000, 0.0000, 0.0000],
         [0.2500, 0.2500, 0.2500,  ..., 0.0000, 1.0000, 1.0000],
         [0.2500, 0.2500, 0.2500,  ..., 1.0000, 0.0000, 0.0000],
         [0.2500, 0.2500, 0.2500,  ..., 0.0000, 0.0000, 0.0000]],

        [[0.2500, 0.2500, 0.2500,  ..., 0.0000, 1.0000, 0.0000],
         [0.2500, 0.2500, 0.2500,  ..., 1.0000, 0.0000, 0.0000],
         [0.2500, 0.2500, 0.2500,  ..., 0.0000, 0.0000, 1.0000],
         [0.2500, 0.2500, 0.2500,  ..., 0.0000, 0.0000, 0.0000]],

        ...,

        [[0.2500, 0.2500, 0.2500,  ..., 0.0000, 0.0000, 0.0000],
         [0.2500, 0.2500, 0.2500,  ..., 0.0000, 1.0000, 1.0000],
         [0.2500, 0.2500, 0.2500,  ..., 0.0000, 0.0000, 0.0000],
     

In [103]:
anndata

In [104]:
atac1

AnnData object with n_obs × n_vars = 1000 × 1000
    obs: 'domain', 'cell_type'
    var: 'chrom', 'chromStart', 'chromEnd', 'highly_variable'
    uns: '__scglue__'
    obsm: 'X_lsi'

In [110]:
for adata, configs in zip(adatas,anndata.data_configs):
    print(adata,configs)
    

rna {'prob_model': 'NB', 'use_highly_variable': True, 'features': ['1110006O24Rik', '1110060G06Rik', '1700001J03Rik', '1700027A07Rik', '1700039E22Rik', '1700047M11Rik', '1700063H06Rik', '1700096K18Rik', '1700120B22Rik', '1700121L03Rik', '1810010H24Rik', '1810019D21Rik', '2010013B24Rik', '2210414B05Rik', '2410004I01Rik', '2510046G10Rik', '2810408I11Rik', '4930417O22Rik', '4930426D05Rik', '4930579K19Rik', '4930597A21Rik', '4932702P03Rik', '4933405L10Rik', '5033403F01Rik', '5330416C01Rik', '5730435O14Rik', '6530403H02Rik', 'A230052G05Rik', 'A730036I17Rik', 'A830018L16Rik', 'A830029E22Rik', 'A930005G22Rik', 'AV099323', 'Abtb2', 'Ace', 'Ackr2', 'Acp5', 'Acsbg1', 'Adap2', 'Adarb2', 'Adgrf2', 'Adrb2', 'Agmat', 'Alcam', 'Aldh1a3', 'Aldoc', 'Alk', 'Ankfn1', 'Aox1', 'Ap1m2', 'Apobr', 'Apoe', 'Appl2', 'Arc', 'Areg', 'Arhgdib', 'Arl9', 'Arx', 'Asgr1', 'Aspa', 'Astn2', 'Atp10a', 'Atp13a5', 'Atp1a2', 'B930094E09Rik', 'BC006965', 'Baiap3', 'Baz1a', 'Bcan', 'Bcas1', 'Bcdin3d', 'Bco2', 'Bdnf', 'Bfsp2',

In [115]:
adatas

{'rna': AnnData object with n_obs × n_vars = 1000 × 28930
     obs: 'domain', 'cell_type'
     var: 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm', 'mean', 'std', 'chrom', 'chromStart', 'chromEnd', 'name', 'score', 'strand', 'thickStart', 'thickEnd', 'itemRgb', 'blockCount', 'blockSizes', 'blockStarts', 'gene_id', 'gene_type', 'mgi_id', 'havana_gene', 'tag'
     uns: 'hvg', 'log1p', 'neighbors', 'pca', 'umap', '__scglue__'
     obsm: 'X_pca', 'X_umap'
     varm: 'PCs'
     layers: 'counts'
     obsp: 'connectivities', 'distances',
 'atac': AnnData object with n_obs × n_vars = 1000 × 1000
     obs: 'domain', 'cell_type'
     var: 'chrom', 'chromStart', 'chromEnd', 'highly_variable'
     uns: '__scglue__'
     obsm: 'X_lsi'}

In [118]:
configs['rna']=anndata.data_configs[0]
configs['atac']=anndata.data_configs[1]

In [123]:
for a, conf in zip(adatas,configs):
    print(a,conf)

rna prob_model
atac use_highly_variable


In [131]:
adatas['rna'].layers['counts']

<1000x28930 sparse matrix of type '<class 'numpy.float32'>'
	with 940822 stored elements in Compressed Sparse Row format>

In [129]:
anndata.data_configs[0].keys()

dict_keys(['prob_model', 'use_highly_variable', 'features', 'use_layer', 'use_rep', 'rep_dim', 'use_batch', 'batches', 'use_cell_type', 'cell_types', 'use_dsc_weight', 'use_obs_names'])

In [130]:
anndata.data_configs[0]['use_layer'] #count data, what difference to adata.X?

'counts'

In [140]:
adatas.keys()

dict_keys(['rna', 'atac'])

In [145]:
import h5py
from anndata._core.sparse_dataset import SparseDataset
arr=atac1.X
isinstance(arr, (h5py.Dataset, SparseDataset))

False

In [148]:
arr[100]#.toarray() 

<1x1000 sparse matrix of type '<class 'numpy.float32'>'
	with 12 stored elements in Compressed Sparse Row format>

In [156]:
np.random.rand(6,4,10)[0].dtype.type

numpy.float64

In [151]:
arr.dtype.type

numpy.float32

In [157]:
atac1.shape[0]

1000

In [158]:
atac1.obsm

AxisArrays with keys: X_lsi

In [161]:
anndata.data_configs[1]["use_rep"]

'X_lsi'

In [162]:
anndata.__getitem__(0)

[tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0.

In [164]:
anndata.shuffle_idx.shape

(2000, 2)

In [171]:
atac1.shape,rna1.shape

((1000, 1000), (1000, 28930))

In [176]:
anndata.data_idx[0].shape

(1000,)

In [178]:
anndata.view_idx.shape

(2000,)

In [179]:
anndata.shuffle_idx

array([[  0,   0],
       [  1,   1],
       [  2,   2],
       ...,
       [997, 997],
       [998, 998],
       [999, 999]])

In [181]:
anndata.getitem_size, anndata.size 

(1, 2000)

In [267]:
index=7090
s=slice(index *1, min((index + 1) * 1, 1000))

In [268]:
shuffle_idx=anndata.shuffle_idx[s].T
shuffle_idx

array([], shape=(2, 0), dtype=int64)

In [189]:
shuffle_pmsk=anndata.shuffle_pmsk[s]
shuffle_pmsk

array([[ True, False]])

In [235]:
for extracted_data in anndata.extracted_data:    
    for idx, data in zip(shuffle_idx, extracted_data):
        print(idx)
        
        print('---------')
        

[3]
---------
[3]
---------
[3]
---------
[3]
---------
[3]
---------
[3]
---------
[3]
---------
[3]
---------
[3]
---------
[3]
---------


In [230]:
len(anndata.extracted_data),anndata.extracted_data

(5,
 ([<1000x800 sparse matrix of type '<class 'numpy.float32'>'
   	with 52349 stored elements in Compressed Sparse Row format>,
   <1000x57 sparse matrix of type '<class 'numpy.float32'>'
   	with 270 stored elements in Compressed Sparse Row format>],
  [array([[ 3.4847796e+00,  1.1418568e+01,  9.0474176e+00, ...,
           -5.3635325e+00,  1.6005282e-01, -1.2033249e+00],
          [ 1.9447638e-01, -4.5760173e-01,  1.2121292e+00, ...,
            7.5727187e-02, -1.0081123e-02, -4.6407688e-02],
          [-1.4876099e+00, -2.0015655e+00,  1.0567182e+00, ...,
           -2.0454429e-01,  2.3665543e-01, -3.0248573e-01],
          ...,
          [ 1.9900976e+01, -1.8118414e+00, -3.2607822e+00, ...,
           -7.5434488e-01,  3.7572000e-03,  6.9936383e-01],
          [-8.4209764e-01, -1.0149227e+00,  2.0335205e+00, ...,
           -5.7078308e-01,  1.1701491e-01, -7.1583271e-02],
          [-1.7389261e+00,  8.8094842e-01,  1.0146681e-02, ...,
           -9.1880590e-01,  3.9583334e-01, -1.6

In [203]:
underscore_extracted=anndata._extract_data(anndata.data_configs)
len(underscore_extracted[0]),len(underscore_extracted[1])

(2, 5)

In [206]:
underscore_extracted[0]

[Index(['a9de3017d45c4cde90dd0253d4f449c1', 'a545b749d9e84498b0815c3176908c50',
        'cbfaa93a26e7435c9e80be574a69ac44', 'b34366c8aa7c41e2ae981b510b898132',
        'f477dd196d6549fabd2ea63fcc50bfb1', '4b8799998448482182b5ad1563cc8c48',
        '15d2ede283754b1b8ccf16bd535d5c7a', '9cbce8d326a04984ac7a58cf41aa6425',
        '38665b408a2947f4aab5593794a5d947', 'ee1a4d9e66aa4baba8fe217644ddc5a9',
        ...
        'b98f521d55494fde8b82392b708df204', '54200976fcd74cd697f8ea6335ffdea9',
        '37687c036d654560a3beee1064903ef9', '8408d2957204432d82ba070b8f2790d6',
        '124abe2a092641888c5027c5514ef55a', 'e53ebb39f4304820ae45f04a298975dd',
        '04bca2126af24a3ea515b85d1bd4cd56', 'edf9f2b817804c9ab3cf800687c19a9e',
        'e9b98790d3f94a6195de50cdc68d2b94', '1ba829538cef45968f4c09fffa9f81d2'],
       dtype='object', length=1000),
 Index(['c8cfb53559804fccbfee65b60d1baf3d', '5d14a197f6f143299e9ddbf43656c1ad',
        '9f4e632d4a5c4084aa1b8900f3ead7a6', '7b1ae48d72d24c9cad8938701

In [212]:
len(underscore_extracted[1][4]),len(underscore_extracted[1][3]),len(underscore_extracted[1][2]),len(underscore_extracted[1][1])

(2, 2, 2, 2)

In [213]:
len(underscore_extracted[1][0])

2

In [218]:
underscore_extracted[1][0]

[<1000x800 sparse matrix of type '<class 'numpy.float32'>'
 	with 52349 stored elements in Compressed Sparse Row format>,
 <1000x57 sparse matrix of type '<class 'numpy.float32'>'
 	with 270 stored elements in Compressed Sparse Row format>]

In [217]:
anndata.extracted_data[0]

[<1000x800 sparse matrix of type '<class 'numpy.float32'>'
 	with 52349 stored elements in Compressed Sparse Row format>,
 <1000x57 sparse matrix of type '<class 'numpy.float32'>'
 	with 270 stored elements in Compressed Sparse Row format>]

In [255]:
for i,d in enumerate(anndata.extracted_data):
    print("{}th element===========".format(i),len(d))
    print(d[0].shape)
    
#lastly append(shuffle_pmsk) True,False 

0th element=========== 2
(1000, 800)
1th element=========== 2
(1000, 100)
2th element=========== 2
(1000,)
3th element=========== 2
(1000,)
4th element=========== 2
(1000,)


In [225]:
shuffle = [2, 0]
extracted = ['apple', 'banana', 'orange', 'grape']

for idx, data in zip(shuffle, extracted):
    print(f"Index: {idx}, Data: {data}")

Index: 2, Data: apple
Index: 0, Data: banana


In [237]:
from scglue.typehint import AnyArray
def _index_array(arr: AnyArray, idx: np.ndarray) -> np.ndarray:
    if isinstance(arr, (h5py.Dataset, SparseDataset)):
        rank = scipy.stats.rankdata(idx, method="dense") - 1
        sorted_idx = np.empty(rank.max() + 1, dtype=int)
        sorted_idx[rank] = idx
        arr = arr[sorted_idx.tolist()][rank.tolist()]  # Convert to sequantial access and back
    else:
        arr = arr[idx]
    return arr.toarray() if scipy.sparse.issparse(arr) else arr

In [240]:
import scipy.sparse
for extracted_data in anndata.extracted_data:    
    for idx, data in zip(shuffle_idx, extracted_data):
#         print(_index_array(data,idx))
        print(idx, data)
        print('---------')

[3]   (0, 69)	1.0
  (0, 95)	1.0
  (0, 110)	1.0
  (0, 114)	1.0
  (0, 116)	1.0
  (0, 138)	2.0
  (0, 171)	1.0
  (0, 331)	1.0
  (0, 334)	1.0
  (0, 367)	3.0
  (0, 381)	1.0
  (0, 395)	3.0
  (0, 401)	1.0
  (0, 402)	3.0
  (0, 403)	1.0
  (0, 406)	3.0
  (0, 408)	1.0
  (0, 410)	5.0
  (0, 418)	5.0
  (0, 429)	1.0
  (0, 439)	1.0
  (0, 474)	1.0
  (0, 479)	3.0
  (0, 490)	6.0
  (0, 514)	2.0
  :	:
  (999, 352)	2.0
  (999, 355)	2.0
  (999, 364)	1.0
  (999, 381)	1.0
  (999, 394)	3.0
  (999, 395)	1.0
  (999, 400)	6.0
  (999, 401)	3.0
  (999, 402)	9.0
  (999, 410)	3.0
  (999, 412)	5.0
  (999, 416)	4.0
  (999, 429)	2.0
  (999, 439)	6.0
  (999, 466)	1.0
  (999, 475)	2.0
  (999, 483)	2.0
  (999, 520)	9.0
  (999, 534)	1.0
  (999, 541)	2.0
  (999, 551)	2.0
  (999, 552)	1.0
  (999, 553)	6.0
  (999, 576)	1.0
  (999, 610)	1.0
---------
[3]   (2, 4)	1.0
  (4, 9)	1.0
  (5, 12)	1.0
  (13, 38)	1.0
  (15, 26)	1.0
  (26, 4)	1.0
  (30, 27)	1.0
  (32, 27)	1.0
  (34, 27)	1.0
  (35, 20)	1.0
  (36, 1)	1.0
  (49, 48)	1.0
  (52

In [244]:
len(anndata.data_idx[0])

1000

In [247]:
anndata.size

2000

In [253]:
idx=[748,401,907,457 ,854, 930, 156,] 
atac1.X[idx]#what next to crc matrix can access line 7888

<7x1000 sparse matrix of type '<class 'numpy.float32'>'
	with 85 stored elements in Compressed Sparse Row format>

In [263]:
anndata[160][0].shape,anndata[160][1].shape,anndata[160][2].shape,anndata[160][3].shape,anndata[160][4].shape

(torch.Size([1, 800]),
 torch.Size([1, 57]),
 torch.Size([1, 100]),
 torch.Size([1, 100]),
 torch.Size([1]))

In [269]:
anndata[7090]

[tensor([], size=(0, 800)),
 tensor([], size=(0, 57)),
 tensor([], size=(0, 100)),
 tensor([], size=(0, 100)),
 tensor([], dtype=torch.int64),
 tensor([], dtype=torch.int64),
 tensor([], dtype=torch.int64),
 tensor([], dtype=torch.int64),
 tensor([]),
 tensor([]),
 tensor([], size=(0, 2), dtype=torch.bool)]

In [275]:
for i in range(len(anndata[2000])):
    data=anndata[2000][i]
    print("{}th data===========".format(i), len(data))
    print(data,)

0th data=========== 0
tensor([], size=(0, 800))
1th data=========== 0
tensor([], size=(0, 57))
2th data=========== 0
tensor([], size=(0, 100))
3th data=========== 0
tensor([], size=(0, 100))
4th data=========== 0
tensor([], dtype=torch.int64)
5th data=========== 0
tensor([], dtype=torch.int64)
6th data=========== 0
tensor([], dtype=torch.int64)
7th data=========== 0
tensor([], dtype=torch.int64)
8th data=========== 0
tensor([])
9th data=========== 0
tensor([])
10th data=========== 0
tensor([], size=(0, 2), dtype=torch.bool)


[tensor([], size=(0, 800)), tensor([], size=(0, 4, 1000)), tensor([], size=(0, 100)), tensor([], size=(0, 100)), tensor([], dtype=torch.int64), tensor([], dtype=torch.int64), tensor([], dtype=torch.int64), tensor([], dtype=torch.int64), tensor([]), tensor([]), tensor([], size=(0, 2), dtype=torch.bool)]
